In [1]:
from map_tools import *

In [2]:
import os

# 设置环境变量
os.environ["OPENAI_API_KEY"] = "sk-GVtO7TUPj9ogA9pECbA7E3961e4946E58c47E437Fc60BcEa"
os.environ["OPENAI_BASE_URL"] = "https://open.xiaojingai.com/v1"



os.environ["OPENAI_API_KEY"] = "sk-sqobvfqyojcartkdkywimrxnhxmrozlpsgvsyrtpecshodid"
os.environ["OPENAI_BASE_URL"] = "https://api.siliconflow.cn/v1/"


In [3]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from map_tools import *  

llm = ChatOpenAI(model="Qwen/Qwen2.5-14B-Instruct")
agent = create_react_agent(model=llm, tools=[get_city_country, get_nearby_locations_bbb, get_nearby_locations_aaa,
                                             get_route_between_cities_bbb, get_route_between_cities_aaa,
                                             get_distance_between_cities_aaa, get_distance_between_cities_bbb,])

KeyboardInterrupt: 

In [4]:
initial_data()

In [ ]:
def eval1():
    """测试获取同一国家(aaa)两个城市之间的距离"""
    cities, routes = load_data()
    aaa_city_ids = list(cities["aaa"].keys())
    chosen_cities = random.sample(aaa_city_ids, 2)
    
    given_para = {"city1_id": chosen_cities[0], "city2_id": chosen_cities[1]}
    query = f"城市{chosen_cities[0]}到城市{chosen_cities[1]}之间的距离是多少？"

    
    def reference_solution(city1_id, city2_id):
        c1 = get_city_country(city1_id)
        c2 = get_city_country(city2_id)
        if c1!=c2:
            distance = get_cross_country_distance(city1_id, city2_id)
        elif c1=="aaa":
            distance = get_distance_between_cities_aaa(city1_id, city2_id)
        else:
            distance = get_distance_between_cities_bbb(city1_id, city2_id)
        return {"distance": distance}
    
    def ref_answer(city1_id, city2_id):
        distance = reference_solution(**given_para)
        return f"距离是{distance}"
    
    def condition(agent_history):
        distance = ref_answer(chosen_cities[0], chosen_cities[1])
        if str(distance) in agent_history:
            return True
        else:
            return False
    answer = ref_answer(**given_para)
    response_conditions = [lambda x: str(answer["distance"]) in x]
    state_conditions = []
    return query, ref_answer, response_conditions, state_conditions

def eval2():
    """测试获取同一国家(bbb)两个城市之间的距离"""
    cities, routes = load_data()
    bbb_city_ids = list(cities["bbb"].keys())
    chosen_cities = random.sample(bbb_city_ids, 2)
    query = f"我想知道bbb国的城市{chosen_cities[0]}和城市{chosen_cities[1]}之间有多远？"

    def ref_answer(city1_id, city2_id):
        return get_distance_between_cities_bbb(city1_id, city2_id)
    
    def condition(agent_history):
        distance = ref_answer(chosen_cities[0], chosen_cities[1])
        if str(distance) in agent_history:
            return True
        else:
            return False
    return query, ref_answer, [condition], chosen_cities

def eval3():
    """测试获取跨国城市之间的距离"""
    cities, routes = load_data()
    aaa_city_ids = list(cities["aaa"].keys())
    bbb_city_ids = list(cities["bbb"].keys())
    city1 = random.choice(aaa_city_ids)
    city2 = random.choice(bbb_city_ids)
    query = f"请告诉我从aaa国的城市{city1}到bbb国的城市{city2}之间的距离是多少？"

    def ref_answer(city1_id, city2_id):
        return get_cross_country_distance(city1_id, city2_id)
    
    def condition(agent_history):
        distance = ref_answer(city1, city2)
        if str(distance) in agent_history:
            return True
        else:
            return False
    return query, ref_answer, [condition], [city1, city2]

def eval4():
    """测试获取aaa国城市的坐标"""
    cities, routes = load_data()
    aaa_city_ids = list(cities["aaa"].keys())
    city_id = random.choice(aaa_city_ids)
    query = f"aaa国的城市{city_id}的地理坐标是什么？"

    def ref_answer(city_id):
        return get_city_coordinates_aaa(city_id)
    
    def condition(agent_history):
        coordinates = ref_answer(city_id)
        # 检查坐标的两个数字是否都出现在回答中
        return all(str(coord) in agent_history for coord in coordinates)
    return query, ref_answer, [condition], city_id

def eval5():
    """测试获取bbb国城市的坐标"""
    cities, routes = load_data()
    bbb_city_ids = list(cities["bbb"].keys())
    city_id = random.choice(bbb_city_ids)
    query = f"请提供bbb国城市{city_id}的经纬度坐标。"

    def ref_answer(city_id):
        return get_city_coordinates_bbb(city_id)
    
    def condition(agent_history):
        coordinates = ref_answer(city_id)
        # 检查坐标的两个数字是否都出现在回答中
        return all(str(coord) in agent_history for coord in coordinates)
    return query, ref_answer, [condition], city_id

def eval6():
    """测试获取aaa国城市附近的地点"""
    cities, routes = load_data()
    aaa_city_ids = list(cities["aaa"].keys())
    city_id = random.choice(aaa_city_ids)
    max_distance = 50
    query = f"列出aaa国城市{city_id}附近{max_distance}单位距离内的所有城市。"

    def ref_answer(city_id, max_distance):
        return get_nearby_locations_aaa(city_id, max_distance)
    
    def condition(agent_history):
        nearby_cities = ref_answer(city_id, max_distance)
        if isinstance(nearby_cities, str):
            return nearby_cities in agent_history
        else:
            # 检查是否至少有一个城市ID出现在回答中
            return any(str(city.name) in agent_history for city in nearby_cities)
    return query, ref_answer, [condition], [city_id, max_distance]

def eval7():
    """测试获取bbb国城市附近的地点"""
    cities, routes = load_data()
    bbb_city_ids = list(cities["bbb"].keys())
    city_id = random.choice(bbb_city_ids)
    max_distance = 30
    query = f"我想知道在bbb国，距离城市{city_id}不超过{max_distance}单位距离的城市有哪些？"

    def ref_answer(city_id, max_distance):
        return get_nearby_locations_bbb(city_id, max_distance)
    
    def condition(agent_history):
        nearby_cities = ref_answer(city_id, max_distance)
        if isinstance(nearby_cities, str):
            return nearby_cities in agent_history
        else:
            # 检查是否至少有一个城市ID出现在回答中
            return any(str(city.name) in agent_history for city in nearby_cities)
    return query, ref_answer, [condition], [city_id, max_distance]

def eval8():
    """测试获取城市所属国家"""
    cities, routes = load_data()
    all_city_ids = list(cities["aaa"].keys()) + list(cities["bbb"].keys())
    city_id = random.choice(all_city_ids)
    query = f"城市{city_id}属于哪个国家？"

    def ref_answer(city_id):
        return get_city_country(city_id)
    
    def condition(agent_history):
        country = ref_answer(city_id)
        return country in agent_history
    return query, ref_answer, [condition], city_id

def eval9():
    """测试获取aaa国两个城市之间的路线信息"""
    cities, routes = load_data()
    aaa_city_ids = list(cities["aaa"].keys())
    chosen_cities = random.sample(aaa_city_ids, 2)
    query = f"请描述从aaa国的城市{chosen_cities[0]}到城市{chosen_cities[1]}的路线，包括步骤数量。"

    def ref_answer(city1_id, city2_id):
        route = get_route_between_cities_aaa(city1_id, city2_id)
        if isinstance(route, str):
            return route
        else:
            return len(route.steps)  # 返回步骤数量
    
    def condition(agent_history):
        steps_count = ref_answer(chosen_cities[0], chosen_cities[1])
        if isinstance(steps_count, str):
            return steps_count in agent_history
        else:
            return str(steps_count) in agent_history
    return query, ref_answer, [condition], chosen_cities

def eval10():
    """测试获取bbb国两个城市之间的路线信息"""
    cities, routes = load_data()
    bbb_city_ids = list(cities["bbb"].keys())
    chosen_cities = random.sample(bbb_city_ids, 2)
    query = f"bbb国的城市{chosen_cities[0]}到城市{chosen_cities[1]}的路线有多少个步骤？"

    def ref_answer(city1_id, city2_id):
        route = get_route_between_cities_bbb(city1_id, city2_id)
        if isinstance(route, str):
            return route
        else:
            return len(route.steps)  # 返回步骤数量
    
    def condition(agent_history):
        steps_count = ref_answer(chosen_cities[0], chosen_cities[1])
        if isinstance(steps_count, str):
            return steps_count in agent_history
        else:
            return str(steps_count) in agent_history
    return query, ref_answer, [condition], chosen_cities

In [69]:

query, ref, _, _ = eval1()
print(query)
response = agent.invoke({"messages": [("user", query)]},)
# response = agent.invoke({"messages": [("user", "中国有多少个地级市")]},)

城市bb64ac83-7c36-407d-985d-9c84dbc1b39c到城市548f9e28-f9c1-482f-9f29-bd53e032c640之间的距离是多少？


In [71]:
def extract_agent_history(response):
    tools = []
    steps = 0
    res = []
    for m in response["messages"]:
        if m.type=="ai":
            res.append(m.content)
        elif m.type=="tool":
            tools.append(m.name)
            steps += 1
    return "\n".join(res), list(set(tools)), steps
        

print(extract_agent_history(response))

('\n城市bb64ac83-7c36-407d-985d-9c84dbc1b39c到城市548f9e28-f9c1-482f-9f29-bd53e032c640之间的距离是87.43公里。', ['get_distance_between_cities_aaa'], 1)
